In [1]:
import numpy as np
import pandas as pd
from lets_plot import *

$\newcommand{\I}{\mathbb{I}}$

### Classification task

Suppose now we want to solve a classification problem and our target values $y\in\{0,1\}$. 
Denote local field or linear predictor $$z = b+w_1x_1^{(i)}+\ldots+w_nx_n^{(i)}=b + W^Tx^{(i)},$$ 

### Logistic function

Logistic function is a special case of sigmoid functions
$$\boxed{\;g(z)=\frac{1}{1+\exp(-z)}=\frac{\exp(z)}{1+\exp(z)}\;}$$

### Activation
Denote $a=g(z)=g\left(b+W^Tx\right)$ and call the resulting value an "activation".


In [2]:
def sigmoid(x,slope=1.0):
    return 1.0/(1.0+np.exp(-slope*x))


In [3]:
def inverse_sigmoid(x):
    return -np.log(1./x -1)

In [4]:
def step_function(x, margin = 0, label=[0,1]):
    return np.where(x >= margin, label[1], label[0])

In [5]:
x = np.linspace(-5, 5, 100)
margin = .6
y = sigmoid(x)
h = step_function(y, margin)
dat = dict({'z': x, 'a': y, 'heavyside': h})
ggplot(dat) + geom_line(aes(x='z', y='a'), color='blue', size=2)  \
        + geom_line(aes(x='z', y='heavyside'), color='black') \
        + geom_segment(x=x[0]-0.5, y=margin, xend=inverse_sigmoid(margin), yend=margin, color='red', linetype='longdash')\
        + labs(title="Sigmoid function and margin") \
        + theme(legend_position='right') + ggsize(500, 300)

### Loss function as a likelihood function
Suppose in the binary classification task
$\begin{align}
&\mathbb{P}\left(y=1 \;|\; x;\, W, b \right) = g(W^Tx+b)=a_1\\
&\mathbb{P}\left(y=0 \;|\; x;\, W, b \right) = 1 - g(W^Tx + b)=a_0,
\end{align}$
where $g(z)$ - **sigmoid** function.

As far as $y\in\{0,1\}$, we can rewrite the above statements in one:
$$\boxed{\;\mathbb{P}\left(y\;|\;x; W, b\right)=g^y(z)\left(1-g(z)\right)^{1-y}\;},$$

where $$z=W^Tx+b$$
Usually it is easier to deal with the logarithm of the likelihood function:
$$\boxed{\;\ell(W, b)=\sum_{i=1}^m\left(y^{(i)}\log g(z^{(i)})+(1-y^{(i)})\log(1-g(z^{(i)}))\right)\;}$$
We can rewrite this expression in the next form:
$$\displaystyle \ell(W, b)=\sum_{i=1}^m\sum_{\ell=0}^1\tilde{y}_{\ell}^{(i)}\log a_{\ell}^{(i)},$$
where $\displaystyle \tilde{y}^{(i)}=\left(\tilde{y}_0^{(i)},\tilde{y}_1^{(i)}\right)$ is one-hot vector: $\displaystyle \tilde{y}_{\ell}^{(i)}=1_{\{y^{(i)}=\ell\}}$

### More than two classes: Softmax function
Let $y^{(i)}$ be **one-hot** vector $y^{(i)}=(0,\ldots,0,1,0,\ldots,0)^T$ and denote 
$$z_{\ell}^{(i)}=b_{\ell}+W_{\ell}^Tx^{(i)}$$
for a class label $\ell\in\{1,\ldots,k\}$ and observation $i=1,\ldots,m$. 

Use softmax function to define activation:
$$a_{\ell}^{(i)}=\frac{\exp(z_{\ell}^{(i)})}{\sum_{j=1}^k\exp(z_j^{(i)})}$$


One can derive the next expression for the log-likelihood finction in this case:
$$\displaystyle \ell(W, b)=\sum_{i=1}^m\sum_{\ell=1}^ky_{\ell}^{(i)}\log a_{\ell}^{(i)}$$

### Stochastic gradient ascent
As far as we are working with the likelihood function - our aim is to find it's maximum and therefore we are going to implement gradient ascent:
$$\boxed{\;W \mapsto W+\eta X^T \left(Y-A\right)\;}$$

### Logistic regression for the multinomial classification
In case of binary classification we had that
$$\log\left(\frac{\mathbb{P}\left(y=0\;|\;x:\theta\right)}{\mathbb{P}\left(y=1\;|\;x;\theta\right)}\right)=0$$
Define the normalizing term $\displaystyle Z=\sum_{\ell=1}^k\exp\left(\theta_{\ell}^Tx\right)$
We can extend logistic regression model to the case of multiple classes:
$$
\log\mathbb{P}\left(y=1\;|\;x\right)=\theta_1^Tx-\log Z\\
\vdots\\
\log\mathbb{P}\left(y=\ell\;|\;x\right)=\theta_{\ell}^Tx-\log Z\\
\vdots\\
\log\mathbb{P}\left(y=k\;|\;x\right)=\theta_k^Tx-\log Z$$
Let's rewrite statements in exponential form:
$$\mathbb{P}\left(y=k\;|\; x\right)=\frac{1}{Z}\exp(\theta_k^Tx)$$
We can obtain expression for $Z$ from the condition (probabilities should sum to one):
$$\sum_{\ell=1}^k\mathbb{P}\left(y=\ell\;|\; x\right)=1$$
and finally we get the softmax function:
$$\boxed{\;\mathbb{P}\left(y=j\;|\; x\right)=\frac{\exp(\theta_j^Tx)}{\sum_{\ell=1}^k\exp(\theta_{\ell}^Tx)}\;}$$

### Hypothesis function
Hypothesis function $h_{\theta}(x)$ for the multinomial classification can be written:
$$h_{\theta}(x)=\left(
\begin{array}{c}
\mathbb{P}(y=1\;|\;x;\theta)\\
\vdots\\
\mathbb{P}(y=\ell\;|\;x;\theta)\\
\vdots\\
\mathbb{P}(y=k\;|\;x;\theta)
\end{array}
\right)=\frac{1}{\sum_{j=1}^k\exp(\theta_j^Tx)}\left(
\begin{array}{c}
\exp(\theta_1^Tx)\\
\vdots\\
\exp(\theta_\ell^Tx)\\
\vdots\\
\exp(\theta_k^Tx)
\end{array}
\right)$$
where $\theta_1,\ldots,\theta_k\in\mathbb{R}^n$ and $\boxed{\;\theta=(\theta_1,\ldots,\theta_k)\in\mathbb{R}^{n\times k}\;}$

### Cost function
The loss function takes the form:
$$J(\theta)=-\sum_{i=1}^m\sum_{\ell=1}^k1\{y^{(i)}=\ell\}\log\left(\frac{\exp(\theta_{\ell}^Tx^{(i)})}{\sum_{j=1}^k\exp(\theta_j^Tx^{(i)})}\right)$$
Predicted class probability:
$$\hat{y}^{(i)}=\mathbb{P}\left(y^{(i)}=\ell\;|\;x^{(i)};\theta\right)=\frac{\exp(\theta_{\ell}^Tx^{(i)})}{\sum_{j=1}^k\exp(\theta_j^Tx^{(i)})}$$


### Cross-entropy loss function
Let $y^{(i)}$ be one-hot vector $y^{(i)}=(0,\ldots,0,1,0,\ldots,0)^T$ and $\hat{y}^{(i)}$ be a model output probability
of a class label $\ell\in\{1,\ldots,k\}$:
$$\hat{y}_{\ell}^{(i)}=\frac{\exp(\theta_{\ell}^Tx^{(i)})}{\sum_{j=1}^k\exp(\theta_{j}^Tx^{(i)})}$$

The cross-entropy function is 
$$\displaystyle J(y, \hat{y})=-\sum_{i=1}^m\sum_{\ell=1}^ky^{(i)}_{\ell}\log \hat{y}_{\ell}^{(i)}$$

Substituiting softmax function for the $\hat{y}$ into the cross-entropy function and 
denoting the resulting function as a function of $\theta$ we obtain:
$$\displaystyle \ell(\theta)=-\sum_{i=1}^m\sum_{\ell=1}^ky^{(i)}\log\left(\frac{\exp(\theta_{\ell}^Tx^{(i)})}{\sum_{j=1}^k\exp(\theta_j^Tx^{(i)})}\right)$$


### Probabilistic approach
The log-likelihood function
$$\ell(\theta)=\sum_{i=1}^m\log p(y^{(i)}\;|\;x^{(i)};\theta)=\sum_{i=1}^m\log \prod_{\ell=1}^k\left(\frac{\exp(\theta_{\ell}^Tx^{(i)})}{\sum_{j=1}^k\exp(\theta_j^Tx^{(i)})}\right)^{1\{y^{(i)}=\ell\}}$$


### Gradient
Take partial derivatives with respect to $\theta_j$:
$$\frac{\partial}{\partial \theta_j}J(\theta)=-\sum_{i=1}^mx^{(i)}\left(1\{y^{(i)}=j\}-\mathbb{P}(y^{(i)}=j\;|\;x^{(i)};\theta)\right)$$
In the vector form:
$$\boxed{\; \nabla_{\theta}J(\theta)=X^T\left(Y-\hat{Y}\right)\;}$$
where $Y\in\mathbb{R}^{m\times k}$ is the matrix of one-hot encoded vectors $y$ and $\hat{Y}=g(X\theta)\in\mathbb{R}^{m\times k}$

### Stochastic gradient ascent
As far as we are working with the likelihood function - our aim is to find it's maximum and therefore we
are going to implement gradient ascent:
$$\boxed{\;\theta \leftarrow \theta+\eta X^T \left(Y-\hat{Y}\right)\;}$$

Control the shapes of the matrices: 
$$X\in\mathbb{R}^{m\times n}$$ 
$$\theta\in\mathbb{R}^{n\times k}$$ 
$$X\theta\in\mathbb{R}^{m\times k}$$
$$\hat{Y}=g(X\theta)\in\mathbb{R}^{m\times k}$$
$$Y\in\mathbb{R}^{m\times k}$$
$$X^T(y-\hat{y})\in\mathbb{R}^{n\times k}$$


In [6]:
def mini_batches(X, y, batch_size):
    m, _ = X.shape
    rand_index = np.random.choice(m, size=m)
    X_, y_ = X[rand_index], y[rand_index]
    pos = 0
    while pos < m:
        X_batch, y_batch = X_[pos:pos+batch_size], y_[pos:pos+batch_size]
        yield (X_batch, y_batch)
        pos += batch_size

In [7]:
def softmax(x):
    a = np.amax(x, axis=1)[:, np.newaxis]
    ex = np.exp(x - a)
    ex_sum = np.sum(ex, axis=1)[:, np.newaxis]
    x = ex / ex_sum
    return x

In [8]:
def one_hot_encoder(labels, n_classes=10):
    encoded = np.zeros((labels.size, n_classes))
    encoded[np.arange(labels.size), labels.astype(int)] = 1
    return encoded

In [9]:
def cross_entropy_loss(output, labels, eps=1e-5):
    logs = np.log(output + eps)
    return np.multiply(-labels, logs).sum(axis=1).mean()


In [36]:
from sklearn.datasets import fetch_openml

In [48]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [53]:
y = np.array(list(map(int, y)))

In [55]:
X = X / 255

In [64]:
def plot_digit(digit, caption=None):
    digit = digit.reshape(28, 28)
    digit = (digit - np.min(digit))/(np.max(digit) - np.min(digit))
    p = ggplot() + geom_image(image_data=digit) + labs(x='', y='') \
        + theme(axis_line='blank', axis_title='blank', axis_ticks='blank', axis_text='blank')
    if caption:
        p += ggtitle(caption)
    return p

In [66]:
random_ind = np.random.randint(low=0, high=X.shape[0], size=100)
digit_examples = X[random_ind]

In [67]:
bunch = GGBunch()
x0, y0 = 0, 0
row = 0
col = 0
width = 50
height = width
step = width + 0
n_cols = 10
for digit in digit_examples:
    if col == n_cols:
        col = 0
        row += 1
    plot = plot_digit(digit)
    bunch.add_plot(plot, x0 + col*step, y0 + row*step, width, height)
    col += 1
bunch.show()

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
np.random.seed(42)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [59]:
def softmax_regression(X, y, n_classes=10, eta=0.001, iter=100):
    _, n = X.shape
    theta = np.zeros((n, n_classes))
    losses = np.zeros(iter)
    for i in range(iter):
        loss = 0
        for batch in mini_batches(X, y, 1000):
            X_, y_ = batch
            y_ = one_hot_encoder(y_, n_classes)
            y_hat = softmax(X_.dot(theta))
            grad = X_.T.dot(y_ - y_hat)
            theta += grad*eta
            loss += cross_entropy_loss(y_hat, y_)
        losses[i] = loss
    return theta, losses

In [60]:
def predict(X, theta):
    y_hat = softmax(X.dot(theta))
    return np.argmax(y_hat, axis=1), y_hat

In [61]:
theta, losses = softmax_regression(X_train, y_train, eta=0.0001, iter=100)

In [62]:
num_epochs = len(losses)
losses_data = pd.DataFrame({'epoch': np.arange(num_epochs),'loss':losses})
other_options = xlim(0, num_epochs) + ggsize(500, 300) + ggtitle('Loss')
ggplot(losses_data) + geom_area(aes(x='epoch',y='loss'), alpha=0.2) + other_options

In [68]:
y_train_model, train_score = predict(X_train, theta)

In [69]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

In [70]:
def gg_confusion_matrix(y, y_hat):
    conf_mat = confusion_matrix(y, y_hat)[::-1]
    confusion_dat = pd.DataFrame(conf_mat)
    observed = confusion_dat.columns.values
    actual = confusion_dat.index.values
    xx, yy = np.meshgrid(actual, observed)
    xx = xx.reshape(-1)
    yy = yy.reshape(-1)
    zz = conf_mat.reshape(-1)
    dat = {'predicted':xx, 'actual':yy[::-1], 'z':zz}
    p = ggplot(dat, aes('predicted', 'actual', fill='z')) \
        + geom_raster() \
        + geom_text(aes(label='z'), color='white')\
        + theme(legend_position='none', axis_ticks='blank', axis_line='blank')\
        + ggsize(600, 600) + scale_x_discrete() + scale_y_discrete()\
        + ggtitle('Confusion matrix')
    return p

In [71]:
gg_confusion_matrix(y_train, y_train_model)

In [72]:
y_test_model, test_score = predict(X_test, theta)

In [73]:
gg_confusion_matrix(y_test, y_test_model)

In [75]:
def f1_score_micro(conf_matrix):
    num_tags = conf_matrix.shape[0]
    score = 0.
    pr, p, r = 0., 0., 0.
    for tag in range(num_tags):
        pr += conf_matrix[tag, tag]
        p += sum(conf_matrix[tag, :])
        r += sum(conf_matrix[:, tag])
    try:
        score = 2 * pr / (p + r)
    except ZeroDivisionError:
        pass
    return score

In [76]:
cm = confusion_matrix(y_test, y_test_model)
score = f1_score_micro(cm)
print(f'F1 score micro: {score}')

F1 score micro: 0.9165714285714286


### Non-targeted attack on the softmax regression
We have our trained model which predicts digits with high accuracy. Is it possibe to fool our high-confident model?
Suppose we show some randomly looking picture and the model says it's digit 7 with high confidence?

In [77]:
def softmax(x):
    if len(x.shape) > 1:
        a = np.amax(x, axis=1)[:, np.newaxis]
        ex = np.exp(x - a)
        ex_sum = np.sum(ex, axis=1)[:, np.newaxis]
        x = ex / ex_sum
    else:
        a = np.amax(x)
        ex = np.exp(x - a)
        ex_sum = np.sum(ex)
        x = ex / ex_sum
    return x


In [78]:
def predict(x, theta):
    y_hat = softmax(theta.T.dot(x))
    return np.argmax(y_hat), y_hat

In [79]:
x_target = X_train[0]

In [80]:
y_pred, score = predict(x_target.T, theta)

In [81]:
bunch = GGBunch()
plot = plot_digit(x_target, 'Predicted digit: '+str(y_pred))
bunch.add_plot(plot, 0, 0)
digit_labels = np.arange(10)
dat = pd.DataFrame({'labels':digit_labels, 'score':score})
plot = ggplot(dat) + geom_histogram(aes('labels', 'score'), stat='identity') + scale_x_discrete()
bunch.add_plot(plot, 500, 0)
bunch.show()

We are going to generate some image that will make our model to predict some certain digit. 
Suppose our target label (which we want our model) make to predict is $\displaystyle y=\left(0,0,1,0,0,0,0,0,0,0\right)^T$ encoding digit 2.
Let us start from the noise image $x$ and then iteratively update it to fit the cross-entropy function 
$\displaystyle J(z)=CE(y, \hat{y})=-\sum_jy_j\log(\hat{y}_j)$, where $\displaystyle \hat{y}=\mathrm{softmax}(z)$ and $z=\theta^Tx$.
Note, that $\theta\in\mathbb{R}^{n\times k}$ is already trained matrix, which we are not going to update during our training.
We can use gradient descent in this case:
$$\boxed{\; x=x-\eta\nabla_xJ(z)\;}$$  

### Differentiation of the cross-entropy function in the softmax regression model
Cross entropy function $\displaystyle CE(y, \hat{y})=-\sum_jy_j\log(\hat{y}_j)$, where $\displaystyle \hat{y}=\mathrm{softmax}(z)$ and $z=\theta^Tx$. 

Consider the function $CE(y, \hat{y})$ as a function of $z$: 
$$\displaystyle J(z)=-\sum_jy_j\log\left(\frac{\exp(z_j)}{\sum_{\ell}\exp(z_{\ell})}\right)$$

Consider
$$
\begin{align*}
    \frac{\partial J(z)}{\partial z_k}=-\frac{\partial}{\partial z_k}\sum_jy_j z_j+\frac{\partial}{\partial z_k}\sum_j y_j\log\sum_{\ell}\exp(z_{\ell})
\end{align*}
$$
Consider the first part of rhs expression:
$$
\begin{align*}
   -\frac{\partial}{\partial z_k}\sum_j y_j z_j=-y_k 
\end{align*}
$$
so, this expression equals 1 if $y_k=1$ which means that $k$ is the actual label, in other cases expression equals zero.

Consider the second part of rhs expression:
$$
\begin{align*}
\sum_j y_j\frac{\partial}{\partial z_k}\log\sum_{\ell}\exp(z_{\ell}) =\\= \sum_j y_j\frac{1}{\sum_{\ell}\exp(z_{\ell})}\frac{\partial}{\partial z_k}\sum_s\exp(z_s)=\\
    = \sum_j y_j\frac{\exp(z_k)}{\sum_{\ell}\exp(z_{\ell})}=\frac{\exp(z_k)}{\sum_{\ell}\exp(z_{\ell})}=\hat{y}_k
\end{align*}
$$
we can get rid of the sum on the last step because $y_i=1$ only if $i=k$, where $k$ is the true label.

And finally we get (if $k$ is the true label), that
$$ 
\begin{align*}
\frac{\partial J(z)}{\partial z_j} = 
\left\{\begin{array}{ll}
\hat{y}_j-y_j,     &  j=k\\
\hat{y}_j,     & j\neq k
\end{array}
\right.
\end{align*}
$$
or in vector form: $\displaystyle \frac{\partial J(z)}{\partial z} = \hat{y}-y$

### Note on matrix differentiation
**The Jacobian**
Let $z(x):\mathbb{R}^{n}\rightarrow\mathbb{R}^k$, then $\displaystyle \frac{\partial z}{\partial x}=\left(
\begin{array}{ccc}
\frac{\partial z_1}{\partial x_1} & \cdots & \frac{\partial z_1}{\partial x_n}\\
\vdots & \ddots & \vdots \\
\frac{\partial z_k}{\partial x_1} & \cdots & \frac{\partial z_k}{\partial x_n}
\end{array}
\right)\in\mathbb{R}^{k \times n}$
Suppose you have a function $z=Wx\in\mathbb{R}^{k\times 1}$, where $W\in\mathbb{R}^{k\times n}$ and $x\in\mathbb{R}^{n\times 1}$.
You can write $$z_i=\sum_{s=1}^nW_{is}x_s$$ 
Consider the $ij$ element of the Jacobian matrix:
$$\frac{\partial z_i}{\partial x_j}=\frac{\partial}{\partial x_j}\sum_{s=1}^nW_{is}x_s=\sum_{s=1}^nW_{is}\frac{\partial x_s}{\partial x_j}=W_{ij}$$
Finally we get:
$$\boxed{\; \frac{\partial Wx}{\partial x}=W \;}$$

### Differentiation with respect to inputs
Find $\frac{\partial J}{\partial x}$ where $J=CE(y, \hat{y})$ in case of softmax regression:
$$
\begin{align*}
    \hat{y}=\mathrm{softmax}\left(\theta^Tx\right)
\end{align*}
$$
Denote
$$
\begin{align*}
    z = \theta^Tx
\end{align*}
$$
We have $$\displaystyle \frac{\partial J}{\partial x}=\frac{\partial J}{\partial z}\frac{\partial z}{\partial x}$$

Differentiate
$$
\begin{align*}
&\frac{\partial J}{\partial z}=\hat{y} - y \\
&\frac{\partial z}{\partial x}=\theta^T
\end{align*}
$$
and finally $$\frac{\partial J}{\partial x}=(\hat{y}-y)\theta^T\in\mathbb{R}^{m\times n}$$

In [82]:
def softmax(x):
    if len(x.shape) > 1:
        a = np.amax(x, axis=1)[:, np.newaxis]
        ex = np.exp(x - a)
        ex_sum = np.sum(ex, axis=1)[:, np.newaxis]
        x = ex / ex_sum
    else:
        a = np.amax(x)
        ex = np.exp(x - a)
        ex_sum = np.sum(ex)
        x = ex / ex_sum
    return x

In [83]:
def adversarial_example(dig, theta, n=784, n_classes=10, eta=0.01, iters=10):
    y = np.zeros(n_classes)
    y[dig] = 1
    x = np.random.normal(loc=0, scale=0.01, size=n)
    for _ in range(iters):
        y_hat = softmax(theta.T.dot(x))
        grad = theta.dot(y_hat - y)
        x -= eta*grad
    return x


In [84]:
def predict(x, theta):
    y_hat = softmax(theta.T.dot(x))
    return np.argmax(y_hat), y_hat

In [85]:
example = adversarial_example(9, theta, iters=100)

In [86]:
y_pred, score = predict(example.T, theta)

In [87]:
bunch = GGBunch()
plot = plot_digit(example, 'Predicted digit: '+str(y_pred))
bunch.add_plot(plot, 0, 0)
digit_labels = np.arange(10)
dat = pd.DataFrame({'labels':digit_labels, 'score':score})
plot = ggplot(dat) + geom_histogram(aes('labels', 'score'), stat='identity') + scale_x_discrete()
bunch.add_plot(plot, 500, 0)
bunch.show()

### Targeted attack
Okay, but is it possible to fool the model with the image which obviously represents some certain digit and make the model to predict another digit with high probability?
Suppose our target label (which we want our model) make to predict is $\displaystyle y=\left(0,0,1,0,0,0,0,0,0,0\right)^T$ encoding digit 2.
Suppose we want the image looking like the above image from the dataset denoted $x_{tgt}\in\mathbb{R}^{n\times 1}$
Our loss function for this task will be
$$J=CE(y, \hat{y})+\alpha\|x-x_{tgt}\|_2^2$$
Our goal is to generate image $x$ fitting this loss function.
The gradient update in this case takes the form
$$\boxed{\;x=x-\eta\left(\nabla_x J+\alpha(x-x_{tgt})\right)\;}$$

In [88]:
def targeted_adversarial_example(dig, x_target, theta, n=784, n_classes=10, eta=0.01, alpha=0.1, iters=10):
    y = np.zeros(n_classes)
    y[dig] = 1
    x = np.random.normal(loc=0, scale=0.01, size=n)
    for _ in range(iters):
        y_hat = softmax(theta.T.dot(x))
        grad = theta.dot(y - y_hat)
        grad -= alpha*(x - x_target)
        x += eta*grad
    return x

In [89]:
example = targeted_adversarial_example(9, x_target, theta, alpha=0.99, iters=100)

In [90]:
y_pred, score = predict(example.T, theta)


In [91]:
bunch = GGBunch()
plot = plot_digit(example, 'Predicted digit: '+str(y_pred))
bunch.add_plot(plot, 0, 0)
digit_labels = np.arange(10)
dat = pd.DataFrame({'labels':digit_labels, 'score':score})
plot = ggplot(dat) + geom_histogram(aes('labels', 'score'), stat='identity') + scale_x_discrete()
bunch.add_plot(plot, 500, 0)
bunch.show()